In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy import genfromtxt
import itertools
import random

Read Embedding File

In [2]:
df=pd.read_csv('emb/karate.emb',skiprows=1,sep=' ',header=None)
df=df.sort_values(by=[0],ascending=True)
emb=df.as_matrix().astype(np.float32)
emb=emb[:,1:]

Read Graph

In [3]:
G=nx.read_edgelist('graph/karate.edgelist',nodetype=int)
edgeLen=len(G.edges())
edges_G=G.edges()
edges_G=np.asarray(edges_G,dtype=int)
np.random.shuffle(edges_G)

Create Egde Feature vectors for positive class

In [4]:
edge_features_1=np.empty(shape=[0,emb.shape[1]])
for edge in edges_G:
    node1=emb[edge[0]-1]
    node2=emb[edge[1]-1]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_1=np.vstack([edge_features_1,hadamard])

In [44]:
G_neg=nx.Graph()
sample_len=len(G.edges())
i=0
E=G.edges()
while(len(G_neg.edges())<sample_len):
    n1=random.choice(G.nodes())
    n2=random.choice(G.nodes())
    i+=1
    e=(n1,n2)
    if(e not in E):
        G_neg.add_edge(*e)

len(G_neg.edges()),i

(77, 84)

In [45]:
i=0
for x in G.edges():
    if(x in G_neg.edges()):
        print x not in G.edges()
        i+=1
print i
print G.edges()
print G_neg.edges()

False
False
False
False
4
[(1, 32), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 11), (1, 12), (1, 13), (1, 14), (1, 18), (1, 20), (1, 22), (2, 3), (2, 4), (2, 8), (2, 14), (2, 18), (2, 20), (2, 22), (2, 31), (3, 4), (3, 33), (3, 8), (3, 9), (3, 10), (3, 14), (3, 28), (3, 29), (4, 8), (4, 13), (4, 14), (5, 11), (5, 7), (6, 7), (6, 11), (6, 17), (7, 17), (9, 34), (9, 33), (10, 34), (14, 34), (15, 33), (15, 34), (16, 33), (16, 34), (19, 33), (19, 34), (20, 34), (21, 33), (21, 34), (23, 33), (23, 34), (24, 33), (24, 34), (24, 28), (24, 26), (24, 30), (25, 32), (25, 26), (25, 28), (26, 32), (27, 34), (27, 30), (28, 34), (29, 32), (29, 34), (30, 33), (30, 34), (31, 33), (31, 34), (32, 33), (32, 34), (33, 34)]
[(1, 17), (1, 14), (1, 13), (1, 30), (2, 9), (3, 27), (3, 11), (3, 6), (4, 10), (4, 11), (4, 20), (4, 5), (4, 31), (5, 6), (5, 10), (5, 13), (5, 15), (5, 16), (5, 22), (5, 23), (5, 25), (5, 26), (5, 27), (6, 32), (6, 6), (6, 16), (6, 22), (6, 26), (6, 28), (6, 31

In [6]:
edge_features_0=np.empty(shape=[0,emb.shape[1]])
for edge in G_neg.edges():
    node1=emb[edge[0]-1]
    node2=emb[edge[1]-1]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_0=np.vstack([edge_features_0,hadamard])

In [7]:
class_0=np.zeros([edgeLen,1])
class_1=np.ones([edgeLen,1])
edge_features_0=np.hstack([edge_features_0,class_0])
edge_features_1=np.hstack([edge_features_1,class_1])
dataset=np.vstack([edge_features_0,edge_features_1])

In [8]:
dataset.shape

(154, 129)

In [9]:
for i in range(10):
    np.random.shuffle(dataset)
X=dataset[:,:dataset.shape[1]-1]
y=dataset[:,dataset.shape[1]-1:]
y=np.reshape(y,[y.shape[0],])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [17]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [18]:
from sklearn.metrics import f1_score,precision_score,recall_score
f1=f1_score(y_test, y_pred, average='macro')
precision=precision_score(y_test, y_pred, average='macro')
recall=recall_score(y_test, y_pred, average='macro')

In [19]:
f1, precision, recall

(0.60648148148148151, 0.60791925465838514, 0.6069230769230769)